
## 23-2 신경망 구성 (1) 개요

신경망이란?   
   
신경계 뉴런이 그물망 형태로 이루고 있는 것을 신경망이라고 부른다   
   
### MNIST Revisited
___
MNIST 이미지 모델 분류기 살펴보기

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test)= mnist.load_data()

# 모델에 맞게 데이터 가공
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped = x_train_norm.reshape(-1, x_train_norm.shape[1]*x_train_norm.shape[2])
x_test_reshaped = x_test_norm.reshape(-1, x_test_norm.shape[1]*x_test_norm.shape[2])

# 딥러닝 모델 구성 - 2 Layer Perceptron
model=keras.models.Sequential()
model.add(keras.layers.Dense(50, activation='sigmoid', input_shape=(784,)))  # 입력층 d=784, 은닉층 레이어 H=50
model.add(keras.layers.Dense(10, activation='softmax'))   # 출력층 레이어 K=10
model.summary()

# 모델 구성과 학습
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train_reshaped, y_train, epochs=10)

# 모델 테스트 결과
test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

/home/jaejin/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jaejin/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jaejin/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jaejin/anaconda3/envs/aiffel/lib/python3

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.4976 - acc: 0.8809
Epoch 2/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.2326 - acc: 0.9344
Epoch 3/10
60000/60000 [==============================] - 2s 32us/sample - loss: 0.1825 - acc: 0.9481
Epoch 4/10
60000/60000 [==============================] - 2s 32us/sample - loss: 0.1524 - 

### 다층 퍼셉트론 Overview
___
   
입력값이 있는 입력층(input layer), 최종 출력값이 있는 출력층(output layer), 그리고 그 사이에 있는 층인 은닉층(hidden layer)이 있습니다.    
  
  보통 그림으로 인공신경망을 표현할 때에는 노드를 기준으로 레이어를 표시해서 3개의 레이어라고 생각할 수 있지만, 실제로는 총 2개의 레이어를 가졌습니다. 레이어 개수를 셀 때에는 노드와 노드 사이의 연결하는 부분이 몇 개 존재하는지 세면 보다 쉽게 알 수 있습니다.--> 이해안감 ㅡㅡ
  
  
  Fully-Connected Neural Network 는 MLP와 다른 용어. 
  
  
### Parameters/Weights
___
입력층 - 은닉층, 은닉층 - 출력층 사이에는 각각 행렬(Matrix)가 존재  
  
  예를들어 입력값이 100개, __은닉 노드가 20개__ 라면 사실 이 입력층_은닉층 사이에는 100*20의 형태를 가진 행렬이 존재합니다. 동일하게, MNIST 데이터처럼 10개의 클래스를 맞추는 문제를 풀기 위해 출력층이 10개의 노드를 가진다면 은닉층-출력층 사이에는 20X10의 형태를 가진 행렬이 존재  
  
  이 행렬들을 Parameter 혹은 Weight라고 부른다.  
  (보통 동일한 뜻이지만 parameter는 bias 노드도 포함한다.)
  
  인접한 레이어 사이에는 아래와 같은 관계가 성립  
  y = W * X + b
    
    MLP 기반 딥러닝 모델을 Numpy로 다시 만들어 보면 다음과 같다

In [2]:
# 입력층 데이터의 모양(shape)
print(x_train_reshaped.shape)

# 테스트를 위해 x_train_reshaped의 앞 5개의 데이터를 가져온다
X = x_train_reshaped[:5]
print(X.shape)

(60000, 784)
(5, 784)


In [3]:
weight_init_std = 0.1
input_size = 784
hidden_size = 50

# 인접 레이어간 관계를 나타내는 파라미터 W를 생성하고 random 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size)
#바이어스 파라미터 b를 생서앟고 Zero로 초기화
b1 = np.zeros(hidden_size)

a1 = np.dot(X,W1) + b1 # 은닉층 출력

print(W1.shape)
print(b1.shape)
print(a1.shape)

(784, 50)
(50,)
(5, 50)


In [4]:
# 첫번째 데이터의 은닉층 출력을 확인해 봅시다.  50dim의 벡터가 나오나요?

a1[0]

array([-0.61152861, -0.81259629, -0.58082487,  0.92820974,  1.31593773,
       -0.39409413, -0.63671789,  0.51146225, -1.20380293,  0.69509845,
       -0.29079939, -0.65399286, -0.14148023, -0.62463569, -1.13579984,
        0.02948256, -0.04376226,  0.21256207,  0.43385903, -1.21647924,
       -1.08928162,  2.00900346,  0.21776318, -0.30800641, -0.09735034,
        0.97377781, -0.10784393, -0.42906225,  0.1111135 ,  0.21217085,
       -0.41380502,  0.58154251, -0.95232087,  0.7123895 ,  0.42795204,
       -1.40021054, -0.8972152 , -0.73149786,  0.14617855, -0.01842886,
        0.41217688,  1.75030309, -0.09521396, -0.89557229,  1.25464859,
       -1.14653327,  0.39375378,  0.07614062,  1.34083456,  1.181767  ])

## 23-3. 신경망 구성 (2) 활성화 함수와 손실 함수

### 활성화 함수 (Activation Functions)
___
MLP의 중요한 구성 요소는 활성화 함수.  
활성화 함수는 보통 비선형 함수를 사용한다. 
비선형 함수를 MLP 안에 포함시키면 모델의 표현력이 좋아진다.( 비선형 함수가 포함되지 않으면 한 층을 가진 MLP와 다른 점이 없음)
   
   활성화 함수의 종류는 다음과 같다 
   1. Sigmoid
   2. Tanh
   3. ReLU

In [5]:
model.add(keras.layers.Dense(50, activation='sigmoid', input_shape=(784)))

TypeError: 'int' object is not iterable

첫번째 은닉층의 출력 a1에 sigmoid를 적용

In [ ]:
# 위 수식의 sigmoid 함수 구현
def sigmoid(x):
    return 1/(1+np.exp(-x))

z1 = sigmoid(a1)
print(z1[0]) # sigmoid의 출력은 모든 엘리먼트가 0에서 1 사이

요즘은 ReLU 함수를 더 많이 사용한다 그 이유는
- Gradient Vanishing 현상이 발생
- exp 함수 사용시 비용이 크다

2. tanh   
    - tanh 함수는 함수의 중심값을 0으로 올며 sigmoid의 최저화 과정이느려지는 문제를 해결
    - gradient vanishing 문제 존재

3. ReLU
    - sigmoid, tanh 함수와 비교 시 학습 빠름.
    - 연산 비용이 크지 않고, 구현이 매우 간단하다

In [ ]:
def affine_layer_forward(X, W, b):
    y= np.dot(X, W) + b
    cache = (X, W, b)
    return y, cache

print('go~')

In [ ]:
input_size = 784
hidden_size = 50
output_size = 10

W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std * np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

a1, cache1 = affine_layer_forward(X, W1, b1)
z1 = sigmoid(a1)
a2, cache2 = affine_layer_forward(z1, W2, b2) # z1이 다시 두번째 레이어의 입력

print(a2[0]) # 최종 출력이 output_size만큼의 백터가 됨

In [ ]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis =0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

In [ ]:
y_hat = softmax(a2)
y_hat[0] # 10개의 숫자 중 하나일 확률

### 손실함수 (Loss Function)
___
__평균제곱오차(MSE: Mean Square Error)__


__교차 엔트로피(cross Entropy)__
- Cross Entropy는 두 확률분포 사이의 유사도가 클수록 작아지는 값
- softmax 값 y_hat은 10개의 숫자 각각의 확률이 대부분 0,1 근처를 오가는 정도


In [ ]:
# 정답 라벨을 One-hot 인코딩 하는 함수
def _change_ont_hot_label(X, num_category):
    T = np.zeros((X.size, num_category))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T

Y_digit = y_train[:5]
t = _change_ont_hot_label(Y_digit, 10)
t

In [ ]:
y_train.shape

In [ ]:
print(y_hat[0])
print(t[0])


In [ ]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis = 1)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

Loss = cross_entropy_error(y_hat, t)
Loss

## 23-4 경사하강법

경사하강법: 각 단계에서의 기울기를 구해 해당 기울기가 가리치는 방향으로 이동하는 방법

Learning rate를된다면, 잘 정하는게 중요하다

1) rate를 매우 크게 했을 경우
경사를 내려가는 Step이 매우 크게 된다면, 기울기의 최소값이 아니라 바로 다시 올라갈 수 있다. overshoothg이라고 한다.

2) 반대로 rate를 매우 작게했을 경우
굉장한 시간이 소요되며, 시간 제한이 있을 경우 경사면이 다 돌기전에 멈춘다.
따라서 미리 cost함수를 실행하며 rate의 비율을 찍어보고 테스트 후에 돌리는 것이 좋다!
  
    
Overfitting  
  
  
머신러닝은 학습을 통해서 모델을 만들다보니 학습 데이터에 딱 맞는 모델을 만들 수 있다.
이 경우에는 실제 테스트 데이터를 적용하면 적용이 잘 되지 않는다.

*Overgitting을 줄이는 방법
  
  1) 트레이닝 데이터가 많으면 좋다
  2) feature 개수를 줄인다.
  3) 정규화(일반화-regulaztion)시킨다.
  
 

In [ ]:
batch_num = y_hat.shape[0]
dy = (y_hat - t) / batch_num
dy # softmax값의 출력으로 loss를 미분한 값

In [ ]:
dw2 = np.dot(z1.T, dy)
db2 = np.sum(dy, axis=0)

In [ ]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [ ]:
dz1 = np.dot(dy, W2.T)
da1 = sigmoid_grad(a1) * dz1
dW1 = np.dot(X.T, da1)
db1 = np.sum(dz1, axis=0)

In [ ]:
learning_rate=0.1

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    return W1, b1, W2, b2

### 23-5 오차역전파법이란?

기울기들을 입력층까지 전달하며 파라미터들을 조정해 나갈 수 있을까요? 이 과정에서 쓰이는 개념이 오차역전파법(Backpropagation)

In [ ]:
def affine_layer_backward(dy, cache):
    X, W, b = cache
    dX = np.dot(dy,W.T)
    dW = np.dot(X.T, dy)
    db = np.sum(dy, axis = 0)
    return dX, dW, db

In [ ]:
# 파라미터 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std*np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

# Forward Propagation
a1, cache1 = affine_layer_forward(X, W1, b1)
z1 = sigmoid(a1)
a2, cache2 = affine_layer_forward(z1, W2, b2)

# 추론과 오차(Loss) 계산
y_hat = softmax(a2)
t = _change_ont_hot_label(Y_digit, 10) # 정답 One-hot 인코딩
Loss = cross_entropy_error(y_hat, t)

print(y_hat)
print(t)
print('Loss:', Loss)

dy = (y_hat - t)/ X.shape[0]
dz1, dW2, db2 = affine_layer_backward(dy, cache2)
da1 = sigmoid_grad(a1)*dz1
dX, dW1, db1 = affine_layer_backward(da1, cache1)

# 경사하강법을 통한 파라미터 업데이트
learning_rate=0.1
W1, b1, W2, b2 = update_params(W1,b1, W2,b2, dW1, db1, dW2, db2, learning_rate)

### 23-6 모델 학습 Step-by-Step

In [ ]:
W1 = weight_init_std*np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std*np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

def train_step(X, Y, W1, b1, W2, b2, learning_rate=0.1, verbose=False):
    a1, cache1 = affine_layer_forward(X, W1, b1)
    z1 = sigmoid(a1)
    a2, cache2 = affine_layer_forward(z1, W2, b2)
    y_hat = softmax(a2)
    t = _change_ont_hot_label(Y,10)
    Loss = cross_entropy_error(y_hat, t)
    
    if verbose:
        print('-------------')
        print(y_hat)
        print(t)
        print('Loss',Loss)
        
    dy = (y_hat-t) / X.shape[0]
    dz1, dW2, db2, = affine_layer_backward(dy, cache2)
    da1 = sigmoid_grad(a1)*dz1
    dX, dW1, db1 = affine_layer_backward(da1, cache1)
    
    W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    
    return W1, b1, W2, b2, Loss

In [ ]:
X = x_train_reshaped[:5]
Y = y_train[:5]

# train_step을 다섯 번 반복 돌립ㄴ디ㅏ.
for i in range(5):
    W1, b1, W2, b2,_ = train_step(X, Y, W1, b1, W2, b2, learning_rate=0.1, verbose=True)

## 23-7 추론 과정 구현과 정확도(Accuracy) 계산

In [ ]:
def predict(W1, b1, W2, b2, x):
    a1 = np.dot(X, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)
    
    return y

In [ ]:
# X = x_train[:100]에 대해 몯레 추론을 시도합니다.

X = x_train_reshaped[:100]
Y = y_test[:100]
result = predict(W1, b1, W2, b2, X)
result[0]

In [ ]:
def accuracy(W1, b1, W2, b2, x,y):
    y_hat = predict(W1, b1, W2, b2, x)
    y_hat = np.argmax(y_hat, axis=1)
    # t = np.argmax(t,axis=1)
    
    accuracy = np.sum(y_hat == y) / float(x.shape[0])
    return accuracy

In [ ]:
acc = accuracy(W1, b1, W2, b2, X, Y)

t = _change_ont_hot_label(Y,10)
print(result[0])
print(t[0])
print(acc)

In [ ]:
x.shape[0]

## 23-8 전체 학습 사이클 수행

In [ ]:
def init_params(input_size, hidden_size, output_size, weight_size_init_std=0.01):
    
    W1 = weight_init_std * np.random.randn(input_size, hidden_size)
    b1 = np.zeros(hidden_size)
    W2 = weight_init_std